In [2]:
import socket
import hmac
import numpy as np
import time

def divide_text(text, block_size_bytes=128):
    blocks = [text[i:i + block_size_bytes] for i in range(0, len(text), block_size_bytes)]
    return blocks

def send_segments(text, sock, ip, port, key, block_size_bytes=128):
    blocks = divide_text(text.encode('utf-8'), block_size_bytes)
    total_payload_sent = 0  # Track goodput (useful data)

    start_time = time.time()  # Start time of transmission

    for i, block in enumerate(blocks):
        mac = hmac.new(key, block, digestmod='sha384').digest()
        message = i.to_bytes(4, 'big') + block + mac  # Prepend block index and append MAC
        total_payload_sent += len(block)  # Count only the useful data
        sock.sendto(message, (ip, port))

    sock.sendto(b'END', (ip, port))  # Indicate end of transmission

    end_time = time.time()  # End time of transmission
    transmission_time = end_time - start_time  # Calculate total transmission time

    print(f"Transmission Time: {transmission_time} seconds")
    print(f"Total Goodput: {total_payload_sent * 8} bits")  # Convert KB to bits

# Example usage
ip = "0.0.0.0"
port = 23422
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

key = b'secret_key'
with open('send.txt', 'r') as file:
    text = file.read()

send_segments(text, sock, ip, port, key)
sock.close()


FileNotFoundError: [Errno 2] No such file or directory: 'send_text.txt'